## Part 0: Introduction to Non Deep Learning Methods

### 1. Linear Classifier

Traditionally, machine learning can be thought of as statistical dataset learning. That is, a system "learns" by building a statistical model from its "experience," which is provided to it in the form of a dataset. Machine learning has been used for a multitude of tasks in areas such as diagnosis, recognition, and planning. We can simplify by categorizing all of these tasks as providing some kind of prediction: in the case of diagnosis, we are predicting something diagnostic, in the case of recognition, we are predicting in the form of recognizing an object or pattern, and in the case of planning, we are predicting the next optimal move in our plan. Broadly, this prediction can come in the form of classification and regression. Mathematically, we can define this as follows:



        

### 2. Support Vector Machine

### 3. Multi-class regression

### 4. Loss functions

#### 4.1 Optimization

### 5. References
<pre>
  [1] Fast.ai (http://course.fast.ai/)  
  [2] CS231N (http://cs231n.github.io/)  
  [3] CS224D (http://cs224d.stanford.edu/syllabus.html)  
  [4] Hands on Machine Learning (https://github.com/ageron/handson-ml)  
  [5] Deep learning with Python Notebooks (https://github.com/fcholletdeep-learning-with-python-notebooks)  
  [6] Deep learning by Goodfellow et. al (http://www.deeplearningbook.org/)  
  [7] Neural networks online book (http://neuralnetworksanddeeplearning.com/)  
</pre>